In [ ]:
#Install Required Libraries
pip install openai langchain faiss-cpu requests beautifulsoup4 python-dotenv

In [ ]:
#setup ENV variable
OPENAI_API_KEY=your_openai_key


In [ ]:
#Use Sample TNFD Report
sample_text = """
The TNFD released new guidelines in April 2025 regarding the financial exposure of insurers to freshwater ecosystem degradation.
This report highlights risks to agricultural insurers in Sub-Saharan Africa and South Asia due to increased drought frequency.
Suggested actions include reassessing reinsurance models and investing in ecosystem protection mechanisms.
"""


In [1]:
#PYTHON CODE
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()
llm = OpenAI(temperature=0.3)

# Define prompt to extract risk and insurance insights
prompt = PromptTemplate(
    input_variables=["text"],
    template="""
You are an insurance risk analyst.
Given the following report text, summarize:
- The climate risk mentioned
- Geographic regions affected
- Insurance lines impacted
- Recommended actions for insurers

Report:
{text}
---
Summary:
""",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the agent
response = chain.run(text=sample_text)
print(response)


In [2]:
#output as JSON
import json
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schemas = [
    ResponseSchema(name="climate_risk", description="Type of risk (e.g., drought, flood)"),
    ResponseSchema(name="regions", description="Regions impacted"),
    ResponseSchema(name="insurance_lines", description="Affected insurance products"),
    ResponseSchema(name="recommendations", description="Actions suggested for insurers"),
]

parser = StructuredOutputParser.from_response_schemas(schemas)

prompt_template = PromptTemplate(
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    template="""
Analyze the following report and extract:
- Climate risk type
- Impacted regions
- Affected insurance lines
- Recommended actions

Report:
{text}

{format_instructions}
""",
)

structured_chain = LLMChain(llm=llm, prompt=prompt_template)
result = structured_chain.run(text=sample_text)

# Convert to Python dict
parsed = parser.parse(result)
print(json.dumps(parsed, indent=2))
